In [ ]:
import pandas as pd
import ast
import re
import os
from dotenv import load_dotenv


In [ ]:
df = pd.read_csv("./euro_cup.csv")
df

# 1. CLEAN DATA

In [ ]:
def strip_data(data, key):
    return [value.strip() for value in data[key]]

def clean_possession(data, key):
    return [int(ps.split('%')[0]) for ps in data[key]]

def load_cleaned_data(data, key, values):
    data[key] = values
    
def format_events(data, key):
    return [list(eval(str(value))) for value in data[key]]

def clean_load_data(data, key, fn):
    cleaned_values = fn(data, key)
    load_cleaned_data(data, key, cleaned_values)

## 1.1 Clean teams name

In [ ]:
clean_load_data(df, 'team_name_home', strip_data)
clean_load_data(df, 'team_name_away', strip_data)

## 1.2 Clean possesion

In [ ]:
clean_load_data(df, 'possession_home', clean_possession)
clean_load_data(df, 'possession_away', clean_possession)

## 1.3 Clean events list

In [ ]:
clean_load_data(df, 'events_list', format_events)

# 2. Extract data

## Common Functions

In [ ]:
def concat_data_frames(df_1, df_2):
    return pd.concat([df_1, df_2])  

def get_event_by_key(team, key_h, key_aw, sub_key):
    return concat_data_frames(scores(f'{team}', f'{key_h}', f'{sub_key}',filter_events_by_home), scores(f'{team}', f'{key_aw}', f'{sub_key}', filter_events_by_aw))

def string_to_number(time):
    return int(time.split("'")[0].strip())

## 2.1 Team names

In [ ]:
teams = list(df['team_name_home'].unique())
teams.sort()

## 2.2 Mean Possession

In [ ]:
def mean_possession_by_team(team):
    pos_h = df[df['team_name_home'] == f'{team}']['possession_home'].sum()
    pos_aw = df[df['team_name_away'] == f'{team}']['possession_away'].sum()
    num_match =  df[(df['team_name_away'] == f'{team}')|(df['team_name_home'] == f'{team}')]['stage'].count()
    mean_pos = (pos_h+pos_aw)/num_match
    return float(mean_pos)

In [ ]:
mean_possession_by_team = [mean_possession_by_team(team) for team in teams]

## 2.3 Goal scores 

In [ ]:
def goals_scored(team):
    scores_h = df[(df['team_name_home'] == f'{team}')]['team_home_score'].sum()
    scores_aw = df[(df['team_name_away'] == f'{team}')]['team_away_score'].sum()
    return int(scores_aw + scores_h)

In [ ]:
goal_by_team = [goals_scored(team) for team in teams]

## 2.4 Goals conceded

In [ ]:
def goals_scored(team):
    scores_h = df[(df['team_name_home'] == f'{team}')]['team_away_score'].sum()
    scores_aw = df[(df['team_name_away'] == f'{team}')]['team_home_score'].sum()
    return int(scores_aw + scores_h)

In [ ]:
goal_conceded_by_team = [goals_scored(team) for team in teams]

## 2.5 Penalties

In [ ]:
def penalties(team):
    penalties_h = int(df[(df['team_name_home'] == f'{team}') & (df['pens_home_score'] != 'False')]['pens_home_score'].sum())
    penalties_w = int(df[(df['team_name_away'] == f'{team}') & (df['pens_home_score'] != 'False')]['pens_away_score'].sum())
    return int(penalties_h + penalties_w)

In [ ]:
penalty_by_team = [penalties(team) for team in teams]

## 2.6  Total shots

In [ ]:
def shots(team):
    shots_h = df[(df['team_name_home'] == f'{team}')]['total_shots_home'].sum()
    shots_aw = df[(df['team_name_away'] == f'{team}')]['total_shots_away'].sum()
    return int(shots_h + shots_aw)

In [ ]:
total_shots_by_team = [shots(team) for team in teams]

## 2.7 Pichici

In [ ]:
def filter_events_by_home(sub_event):
    return sub_event['event_team'] == 'home'

def filter_events_by_aw(sub_event):
    return sub_event['event_team'] == 'away'

def parse_events(lst, fn):
    events_h_f = []
    for event in lst:
        for sub_event in event:
            if(fn(sub_event)):
                    events_h_f.append(sub_event)
    return events_h_f
            
def events_by_team(team, key):
    return df[df[key] == f'{team}']['events_list'].values

def events_filter_parsed(team, key, fn):
    return parse_events(events_by_team(f'{team}', f'{key}') , fn)

def to_data_frame(values):
    return pd.DataFrame(values)

def sub_event_by(sub_event_df, key):
    return sub_event_df[sub_event_df['event_type'] == f'{key}']

def top_scorer(lst):
    try:
        return lst['action_player_1'].value_counts().idxmax()
    except ValueError:
        return ''

def data_frame(team, key, fn):
    return to_data_frame(events_filter_parsed(f'{team}', f'{key}', fn))

def scores(team, key,  sub_key, fn):
    return sub_event_by(data_frame(f'{team}', f'{key}', fn), f'{sub_key}')

In [ ]:
top_scores_by_team = [top_scorer(get_event_by_key(f'{team}', 'team_name_home', 'team_name_away','Goal')) for team in teams]

## 2.8 Color cards

In [ ]:
def cards_by_color(color_card):
    return [int(get_event_by_key(f'{team}', 'team_name_home', 'team_name_away', color_card)['event_type'].count()) for team in teams]

In [ ]:
yellow_cards_by_team = cards_by_color('Yellow card')
red_cards_by_team = cards_by_color('Red card')

## 2.9 Extra time

In [ ]:
def df_event_time_formated(team, key, fn):
    edf = data_frame(team, key, fn)
    edf['event_time'] = [string_to_number(time) if (time != False) else 0 for time in edf['event_time']]
    return edf
   
def count_match_ex_time(e_h_df, e_aw_df):
    return len(e_h_df[e_h_df['event_time'] > 100]) + len(e_aw_df[e_aw_df['event_time'] > 100])

def get_events_by_match(team, key):
    events_by_match_h = list(events_by_team(f'{team}', f'{key}'))

def count_match_with_extra_time(match):
    count = 0
    for events in match:
        df_by_match = to_data_frame(events)
        df_by_match['event_time'] = [string_to_number(time) if (time != False) else 0 for time in df_by_match['event_time']]
        if len(df_by_match[df_by_match['event_time'] > 110]) > 0 : count +=1
    return count
    
def get_extra_time(team):
    events_by_match_h = list(events_by_team(f'{team}', 'team_name_home'))
    events_by_match_aw = list(events_by_team(f'{team}', 'team_name_away'))
    
    count_home = count_match_with_extra_time(events_by_match_h)
    count_away = count_match_with_extra_time(events_by_match_aw)
  
    return count_home + count_away

extra_times_by_team = [get_extra_time(team) for team in teams]

# 3 Extract data from Apis

In [ ]:
import requests
import json
import time
load_dotenv()

def get_api_token(key):
    return os.getenv(key)

def request_to_api(url, api_token):
    response = requests.get(url, headers={'X-Auth-Token': api_token})
    return response.json()

def json_to_dic(value):
    data = json.dumps(value)
    return json.loads(data)

## 3.1 Players From API

In [ ]:
api_token = get_api_token('FOOTBALL_DATA_TOKEN')
response = request_to_api('https://api.football-data.org/v2/competitions/2018/teams', api_token)

In [ ]:
dc = json_to_dic(response)
teams_id = [{'name': team['shortName'], 'id': team['id']} for team in dc['teams']]

In [ ]:
def get_players_by_team(team):
    team_id = team['id']
    url_teams = f'http://api.football-data.org/v2/teams/{team_id}'
    res_teams = requests.get(url_teams, headers={'X-Auth-Token': api_token})
    res_teams_json = res_teams.json()
    dc = json.loads(json.dumps(res_teams_json))
    team.update({'squad': dc['squad']})
    return team

In [ ]:
def get_all_players():
    section_low = 0
    section_gt = 8
    all_players = []
    for _ in range(3):
        all_players += [get_players_by_team(team) for team in teams_id[section_low:section_gt]]
        time.sleep(60)
        section_low += 8
        section_gt += 8
    return all_players

In [ ]:
all_players = get_all_players()

In [ ]:
def clean_all_players(squads):
    all_squad_cleaned = []
    for squad in squads:
        squad_cleaned =  { 'name': squad['name'], 'id': squad['id'], 'squad': squad['squad'] }
        all_squad_cleaned.append(squad_cleaned)
    return all_squad_cleaned

In [ ]:
squad_cleaned = clean_all_players(all_players)

# Create stadistics document

In [ ]:
def add_stadistic(data, idx):
    return data[idx]

def crate_stadistics_document(idx):
    return {
        "team": add_stadistic(teams, idx),
        "mean_possession": add_stadistic(mean_possession_by_team, idx),
        "goal_scored": add_stadistic(goal_by_team, idx),
        "goal_conceded": add_stadistic(goal_conceded_by_team, idx),
        "penalties": add_stadistic(penalty_by_team, idx),
        "shots": add_stadistic(total_shots_by_team, idx),
        "top_scored": add_stadistic(top_scores_by_team, idx),
        "yellow_cards": add_stadistic(mean_possession_by_team, idx),
        "red_cards": add_stadistic(red_cards_by_team, idx),
        "extra_times": add_stadistic(extra_times_by_team, idx),
    }

In [ ]:
teams_document = [crate_stadistics_document(idx) for idx, value in enumerate(teams)]

## Clean team_stadisctis

In [ ]:
def remove_id_from_squads(players):
    for squad in players:
        if '_id' in squad.keys():
            del squad['_id']
    return players

## Save Stadistics in Database

In [ ]:
response = requests.post('http://localhost:8000/stadistics/', json=teams_document)

## Save Players in Database

In [ ]:
response = requests.post('http://localhost:8000/squads/', json=squad_cleaned)

# Web Scraping players stadistics

In [ ]:
from bs4 import BeautifulSoup

def request_to_api_text(url, api_token):
    response = requests.get(url, headers={'X-Auth-Token': api_token})
    return response.text

In [ ]:
res = request_to_api_text('https://es.uefa.com/uefaeuro-2020/teams/players/250063984--leroy-sane/statistics/', api_token)

## Player stadistics Midfielder

In [ ]:
url_teams = 'https://es.uefa.com/uefaeuro-2020/teams/'
res = request_to_api_text(url_teams, api_token)

In [ ]:
html_tams = BeautifulSoup(res)

In [ ]:
def get_teams_id():
    teams_url = html_tams.select('div.history-clubs-list > a', href=True)
    all_teams_url = [team_url['href'] for team_url in teams_url]
    teams_id = [re.search('[0-9]+--[a-z]+', team_url).group() for team_url in all_teams_url ]
    return [f'https://es.uefa.com/uefaeuro-2020/teams/{team_id}/squad/' for team_id in teams_id]

In [ ]:
def get_players_id(team_squad_url):
    res_team_squad = request_to_api_text(team_squad_url, api_token)
    players_html = BeautifulSoup(res_team_squad)
    players = players_html.select('td.squad--player-headshot a')
    players_id =  [f'https://es.uefa.com{player["href"]}' for player in players]
    players_id = list(dict.fromkeys(players_id))
    return players_id

In [ ]:
def get_html_player(complete_player_url):
    res_player = request_to_api_text(complete_player_url, api_token)
    return BeautifulSoup(res_player)

def get_player_position(complete_player_url):
    html_player = get_html_player(complete_player_url)
    position_html = html_player.select('span.player-header_category')[0]
    return position_html.get_text()

In [ ]:
def extract_by_reg(pattern, string):
    return re.sub(pattern,'', string)

def stadistics_key(idx):
    keys = ['matchs', 'min', 'goals', 'cards', 'foul', 'passing_precision', 'shots', 'assists', 'duels', 'recovered_balls', 'max_speed', 'distance']
    return keys[idx]

def parse_to_key_value(sub_str, idx):
    return (stadistics_key(idx), extract_by_reg('[a-zA-Z]', sub_str).strip())
    
def scrape_statdistics(document):
    table = document.select('div.box-content.clearfix')[0]
    stadistic_player_table = table.select('div.field.field-xs-small')[0:12]
    return [e.get_text(strip=True) for e in stadistic_player_table]

def format_value(dic, key, pattern):
    val = dic[key]
    res = re.search(pattern, val)
    return res.group()
    
def replace_dc_data(dc, key, value):
    dc[key] = value
    return dc

def replace_specific_data(dc):
    try:
        replace_dc_data(dc,'passing_precision', format_value(dc, 'passing_precision', '[0-9]{2}%').split('%')[0])
        replace_dc_data(dc,'max_speed', format_value(dc, 'max_speed', '[0-9]+,?[0-9]*').replace(',','.'))
        replace_dc_data(dc,'distance', format_value(dc, 'distance', '[0-9]+').replace(',','.'))
        replace_dc_data(dc,'min', format_value(dc, 'min', '[0-9]+'))
    except:
        return dc
    return dc
    
def parse_to_float(dc):
    for key, value in dc.items():
        dc[key] = float(value)
    return dc

def create_stadistics_dict(document):
    keys_values = [parse_to_key_value(sub_str, idx) for idx, sub_str in enumerate(scrape_statdistics(document))]
    return dict(keys_values)

def have_stadistics(html_player):
    table = html_player.select('div.box-content.clearfix')[0]
    return len(table.select('div.field.field-xs-small')[0:12]) > 0

def get_player_stadistics(html_player):
    if have_stadistics(html_player):
        dc = create_stadistics_dict(html_player)
        dc = replace_specific_data(dc)
        dc = parse_to_float(dc)
        return dc

In [ ]:
def get_html_url_players(complete_players_url):
    return [(get_html_player(player_url), player_url) for player_url in complete_players_url]

In [ ]:
def get_all_player_stadistics(html_url_players):
    return [get_player_stadistics(html_url_player[0]) for html_url_player in html_url_players if get_player_position(html_url_player[1]) == 'Centrocampista' ]

In [ ]:
all_teams_url = get_teams_id()

In [ ]:
all_players_id = [get_players_id(team_id) for team_id in all_teams_url]

In [ ]:
flat_all_players_id = [item for sublist in all_players_id for item in sublist]

In [ ]:
all_html_url_players_tuple= get_html_url_players(flat_all_players_id)

In [ ]:
list_of_stadistics = get_all_player_stadistics(all_html_url_players_tuple)